# Prediction After Fine-Tuning - Simple Example.

Date
: 06/03/2025

Author
: Kaya Arkin (Stu No. 2105361)

Copyright
: Swansea University

### **Context**
This jupyter notebook is authored by Kaya Arkin (Stu No. 2105361) as part of their Swansea University final year project *"Exploratory Research on Explainable LLMs (Airbus AI Research)"*. The project is supervised by Mark Hall, an employee at the Airbus AI Research Department, and Bertie Muller, a university assigned supervisor. The project aims *"to provide an insightful set of findings and recommendations on fine-turned local explanations for LLMs that can utilised as resource for future explainability implementations"* tailored towards Airbus AI Research.

Please ensure you have read `README.txt` before continuing.

---

### **Code Explanation**

THe jupyter notebook provides an example of the fine-tuned LLM predicting a part failure for a given report. In this example, the fine-tuned model is loaded, an input is tokenised and fed into the model which outputs a predicted response.

---


## 1. Imports

In [23]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

---

## 2. Hardware Setup

In [24]:
## Hardware Setup Constants
CPU_DEVICE_NAME = "cpu"
GPU_DEVICE_NAME = "cuda"

In [25]:
## Set processing to GPU if available
if torch.cuda.is_available():
    device = GPU_DEVICE_NAME

    ## Empty GPU VRAM
    torch.cuda.empty_cache()

    print("GPU processing enabled.")

else:
    device = CPU_DEVICE_NAME
    print("GPU processing not available.")

GPU processing enabled.


---

## 3. Preprocessing Input

In [26]:
## Preprocessing Constants
MODEL_PATH = "./t5_finetuned_airline_incidents"
INPUT_FORMAT = "Report: {report}"

In [27]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH).to(device)

In [28]:
from transformers import BatchEncoding ## This import is only used for python annotations, not implementation

def tokenise_input(report: str) -> BatchEncoding:
    input_text = INPUT_FORMAT.format(report=report)
    return tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)


---

## 4. Generate Prediction

In [29]:
## Generating Prediction Constants
EXAMPLE_REPORT = "THE FLIGHT CREW REPORTED OF A BAGGAGE/FUEL DOOR CAS MESSAGE. OPERATIONS WERE CONTINUED PER MEL 52-4. THE MAINTENANCE TEAM TROUBLESHOT AND FOUND THE FUEL DOOR MICROSWITCH TO BE DEFECTIVE. THE SWITCH WAS REPLACED WITH NEW AND RIGGED. OPERATIONS WERE SATISFACTORY AND THE AIRCRAFT WAS RETURNED TO SERVICE."

In [30]:
def get_predicted_part_failure(report: str) -> str:
    inputs = tokenise_input(report)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=128)

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [31]:
print("Report:", EXAMPLE_REPORT)
print(get_predicted_part_failure(EXAMPLE_REPORT))

Report: THE FLIGHT CREW REPORTED OF A BAGGAGE/FUEL DOOR CAS MESSAGE. OPERATIONS WERE CONTINUED PER MEL 52-4. THE MAINTENANCE TEAM TROUBLESHOT AND FOUND THE FUEL DOOR MICROSWITCH TO BE DEFECTIVE. THE SWITCH WAS REPLACED WITH NEW AND RIGGED. OPERATIONS WERE SATISFACTORY AND THE AIRCRAFT WAS RETURNED TO SERVICE.
Part Failure: FUEL DOOR DEFECTIVE


In [95]:
new_report = "THE FLIGHT CREW REPORTED OF A BAGGAGE/FUEL DOOR CAS MESSAGE. OPERATIONS WERE CONTINUED PER MEL 52-4. THE MAINTENANCE TEAM TROUBLESHOT AND FOUND THE FUEL DOOR MICROSWITCH TO BE DEFECTIVE. THE SWITCH WAS REPLACED WITH NEW AND RIGGED. OPERATIONS WERE SATISFACTORY AND THE AIRCRAFT WAS RETURNED TO SERVICE."
print("Report:", new_report)
print(get_predicted_part_failure(new_report))

Report: THE FLIGHT CREW REPORTED OF A BAGGAGE/FUEL DOOR CAS MESSAGE. OPERATIONS WERE CONTINUED PER MEL 52-4. THE MAINTENANCE TEAM TROUBLESHOT AND FOUND THE FUEL DOOR MICROSWITCH TO BE DEFECTIVE. THE SWITCH WAS REPLACED WITH NEW AND RIGGED. OPERATIONS WERE SATISFACTORY AND THE AIRCRAFT WAS RETURNED TO SERVICE.
Part Failure: FUEL DOOR DEFECTIVE
